In [111]:
# !pip install -U -q segmentation-models
# !pip install -q tensorflow==2.1
# !pip install -q keras==2.3.1
# !pip install -q tensorflow-estimator==2.1.

# ## Imports libs
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ["SM_FRAMEWORK"] = "tf.keras"

# from tensorflow import keras
# import segmentation_models as sm

In [112]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import Counter
from keras.preprocessing import sequence
import random

In [113]:
def create_letter_mapping(all_words):
    letter_li = []
    
    # Get unique letters in the data
    for word in all_words:
        for letter in word:
            letter_li.append(letter)
    
    # order letters by most common
    most_common = Counter(letter_li).most_common()
    letter_li = []
    for elem in most_common:
        letter_li.append(elem[0])
    
    # Map each letter to an integer
    letter_dict = {}
    for idx, letter in enumerate(letter_li):
        if letter_dict != '':
            letter_dict[letter] = idx + 1
        else:
            letter_dict[letter] = 0
    
    return letter_dict

def letter_to_number(word_li, letter_dict):
    return_li = []
    for letter in word_li:
        return_li.append(letter_dict[letter])
        
    return return_li

In [114]:
lan_df = pd.read_csv('language_data.csv')

In [115]:
lan_df = lan_df[lan_df['Group'] != 'Other']

In [116]:
lan_df.reset_index(inplace=True,drop=True)

In [117]:
group_dict = {}
for idx, value in enumerate(lan_df.Group.unique()):
    group_dict[value] = idx
    
lan_df['group_int'] = lan_df['Group'].apply(lambda x: group_dict[x])

In [118]:
lan_df['letter_li'] = lan_df['word'].apply(lambda x: list(x))

In [119]:
map_dict = create_letter_mapping(lan_df['word'].values)

In [120]:
lan_df['letter_ints'] = lan_df['letter_li'].apply(lambda x: letter_to_number(x, map_dict))

In [121]:
lan_df.iloc[0, :]

word                                 abs brake
origin                                     deu
full_name                               German
Group                                 Germanic
word_len                                     9
group_int                                    0
letter_li          [a, b, s,  , b, r, a, k, e]
letter_ints    [3, 16, 8, 27, 16, 7, 3, 21, 1]
Name: 0, dtype: object

In [122]:
SHAPE = lan_df.shape[0]

In [123]:
word_arr = np.empty(SHAPE, dtype=list)
origin_arr = np.empty(SHAPE, dtype=int)

In [124]:
for i in range(SHAPE):
    word_arr[i] = lan_df.loc[i, 'letter_ints']
    origin_arr[i] = lan_df.loc[i, 'group_int']

In [125]:
# pad array so that all words have length of 25
word_arr = sequence.pad_sequences(word_arr, 25)

In [126]:
# Develop random index for train test split
random.seed(0)
rng = np.arange(SHAPE)
random.shuffle(rng)

In [127]:
train_len = int(SHAPE * 0.8)
train_idx = rng[:train_len]
test_idx = rng[train_len:]

In [128]:
X_train, y_train = word_arr[train_idx], origin_arr[train_idx]
X_test, y_test = word_arr[test_idx], origin_arr[test_idx]

In [129]:
len(map_dict)

53

In [130]:
len(group_dict)

5

In [131]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(map_dict) + 1, 10),
    tf.keras.layers.LSTM(10),
    tf.keras.layers.Dense(len(group_dict), activation="softmax")
])

In [132]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Train on 14935 samples, validate on 3734 samples
Epoch 1/10
14935/14935 [==============================] - 11s 750us/sample - loss: 0.5956 - acc: 0.8625 - val_loss: 0.4565 - val_acc: 0.8618
Epoch 2/10
14935/14935 [==============================] - 9s 584us/sample - loss: 0.4385 - acc: 0.8644 - val_loss: 0.4296 - val_acc: 0.86310.4378 - acc: 0.86
Epoch 3/10
14935/14935 [==============================] - 9s 573us/sample - loss: 0.4223 - acc: 0.8648 - val_loss: 0.4167 - val_acc: 0.8640
Epoch 4/10
14935/14935 [==============================] - 9s 590us/sample - loss: 0.4143 - acc: 0.8653 - val_loss: 0.4072 - val_acc: 0.8664
Epoch 5/10
14935/14935 [==============================] - 9s 599us/sample - loss: 0.4076 - acc: 0.8670 - val_loss: 0.4007 - val_acc: 0.8698
Epoch 6/10
14935/14935 [==============================] - 9s 586us/sample - loss: 0.4012 - acc: 0.8698 - val_loss: 0.3986 - val_acc: 0.8698
Epoch 7/10
14935/14935 [==============================] - 9s 596us/sample - loss: 0.3962 - a

In [133]:
word_arr

array([[ 0,  0,  0, ...,  3, 21,  1],
       [ 0,  0,  0, ...,  2, 19, 14],
       [ 0,  0,  0, ..., 17,  1,  4],
       ...,
       [ 0,  0,  0, ...,  5,  2, 10],
       [ 0,  0,  0, ...,  7,  6,  9],
       [ 0,  0,  0, ..., 17, 11, 12]])

In [134]:
lan_df.head()

,word,origin,full_name,Group,word_len,group_int,letter_li,letter_ints
0,abs brake,deu,German,Germanic,9,0,"[a, b, s, , b, r, a, k, e]","[3, 16, 8, 27, 16, 7, 3, 21, 1]"
1,aivd,nld,Dutch;Flemish,Germanic,4,0,"[a, i, v, d]","[3, 2, 19, 14]"
2,aachen,deu,German,Germanic,6,0,"[a, a, c, h, e, n]","[3, 3, 10, 17, 1, 4]"
3,aarhus,dan,Danish,Germanic,6,0,"[a, a, r, h, u, s]","[3, 3, 7, 17, 11, 8]"
4,ababil,ara,Arabic,Arabic,6,1,"[a, b, a, b, i, l]","[3, 16, 3, 16, 2, 9]"


In [171]:
lan_df[(lan_df['Group'] == 'Japanese') & (lan_df['word'].str.startswith('g'))]

,word,origin,full_name,Group,word_len,group_int,letter_li,letter_ints
9209,gasshohineri,jpn,Japanese,Japanese,12,4,"[g, a, s, s, h, o, h, i, n, e, r, i]","[18, 3, 8, 8, 17, 6, 17, 2, 4, 1, 7, 2]"
9233,geisha,jpn,Japanese,Japanese,6,4,"[g, e, i, s, h, a]","[18, 1, 2, 8, 17, 3]"
9286,genkan,jpn,Japanese,Japanese,6,4,"[g, e, n, k, a, n]","[18, 1, 4, 21, 3, 4]"
9289,genro,jpn,Japanese,Japanese,5,4,"[g, e, n, r, o]","[18, 1, 4, 7, 6]"
9345,geta,jpn,Japanese,Japanese,4,4,"[g, e, t, a]","[18, 1, 5, 3]"
9360,gingko,jpn,Japanese,Japanese,6,4,"[g, i, n, g, k, o]","[18, 2, 4, 18, 21, 6]"
9441,gokkun,jpn,Japanese,Japanese,6,4,"[g, o, k, k, u, n]","[18, 6, 21, 21, 11, 4]"
9641,gunbai,jpn,Japanese,Japanese,6,4,"[g, u, n, b, a, i]","[18, 11, 4, 16, 3, 2]"
9669,gyotaku,jpn,Japanese,Japanese,7,4,"[g, y, o, t, a, k, u]","[18, 15, 6, 5, 3, 21, 11]"


In [137]:
group_dict

{'Germanic': 0, 'Arabic': 1, 'Latin': 2, 'Greek': 3, 'Japanese': 4}

In [172]:
[int(elem * 100) for elem in model.predict(word_arr[9669:9669 + 1])[0]]

[19, 3, 64, 2, 11]